# Chapter 8: Stacks and Queues

### Notes:
- For an `list`-based stack `s`:
    * push -> `s.append()`
    * pop -> `s.pop()`
    * peek -> `s[-1]`


- For a `deque`-based queue `q`:
    * enqueue -> `q.append()`
    * dequeue -> `q.popleft()`

In [51]:
from collections import namedtuple, deque

class Stack:
    """
    Stack wrapper over a `list`
    """
    # Constructor
    def __init__(self):
        self._stack = []
        
    # Methods
    def push(self, item): 
        self._stack.append(item)
    
    def pop(self):
        if self._is_empty():
            raise IndexError("Cannot pop from an empty stack.")
        else:
            return self._stack.pop()
    
    def peek(self):
        if self._is_empty():
            raise IndexError("Cannot peek into an empty stack.")
        else:
            return self._stack[-1]
    
    def size(self):
        return len(self._stack)
        
    # Functions
    def _is_empty(self):
        return len(self._stack) == 0
    
    
class Queue:
    """
    Queue wrapper over a `deque`
    """
    # Constructor
    def __init__(self):
        self._q = deque()
        
    # Methods
    def enqueue(self, item):
        self._q.append(item)
    
    def dequeue(self):
        self._q.popleft()
    
    def max(self):
        return max(self._q)

## 8.1  Implement a stack with max() API

In [3]:
class StackWithMax:
    """
    Stack wrapper over a list with `max`
    """
    StackItem = namedtuple('StackItem', ('item', 'max'))
    
    # Constructor
    def __init__(self):
        self._stack = []
        
    # Methods
    def push(self, item):
        self._stack.append(self.StackItem(item=item, 
                                          max=item if self._is_empty() 
                                                   else max(item, self.max())))
    
    def pop(self):
        if self._is_empty():
            raise IndexError("Cannot pop from an empty stack.")
        else:
            return self._stack.pop().item
    
    def peek(self):
        if self._is_empty():
            raise IndexError("Cannot peek into an empty stack.")
        else:
            return self._stack[-1].item
    
    def max(self):
        if self._is_empty():
            raise IndexError("Cannot peek into an empty stack.")
        else:
            return self._stack[-1].max
        
    # Functions
    def _is_empty(self):
        return len(self._stack) == 0

In [4]:
# Tests
s = StackWithMax()
s.push(1)
assert s.peek() == 1
assert s.pop() == 1

try:
    s.max()
except IndexError:
    pass

s.push(10)
s.push(11)
assert s.max() == 11
assert s.pop() == 11
assert s.max() == 10
assert s.pop() == 10

Time Complexity is `O(1)` for each of the above operations.

Space Complexity is `O(n)`.

##  8.2  Evaluate RPN Expressions

In [5]:
def evaluate_rpn(exp):
    """
    Returns the result of an arithmetic expression in Reverse Polish Notation
    """
    OPERATORS = {
        "+": lambda y, x: x + y,
        "-": lambda y, x: x - y,
        "x": lambda y, x: x * y,
        "/": lambda y, x: int(x / y)
    }
    s = Stack()
    for t in exp.strip().split(","):
        if t not in OPERATORS:
            s.push(int(t))
        else:
            s.push(OPERATORS[t](s.pop(), s.pop()))
    return s.pop()

# Tests
assert evaluate_rpn("3,4,+,2,x,1,+") == 15
assert evaluate_rpn("1729") == 1729

Time and Space Complexities are `O(n)`, where `n` is the length of the RPN string.

## 8.3  Test a string over `"{,},(,),[,]"` for well-formedness

In [50]:
def is_well_formed(exp):
    """
    Returns True iff the given expression is well-formed
    """
    s, exp_map = Stack(), {")": "(", "]": "[", "}": "{"}
    for c in exp:
        if c not in exp_map:
            s.push(c)
        else:
            if s.size() == 0 or s.pop() != exp_map[c]:
                return False
    return s.size() == 0

# Tests
assert is_well_formed("([]){()}")
assert is_well_formed("[()[]{()()}]")
assert not is_well_formed("{)")
assert not is_well_formed("[()[]{()()")
assert not is_well_formed("}")
assert not is_well_formed("{")

Time and Space Complexity: `O(n)`

## 8.4  Implement a circular queue

In [39]:
class CircularQueue:
    """
    Dynamic Circular Queue wrapper over a `list`
    """
    RESIZE_FACTOR = 2
    
    def __init__(self, capacity):
        self._q = [None] * capacity
        self._size = 0
        self._head = 0
        self._tail = 0
    
    def enqueue(self, item):
        if self._size == len(self._q):
            # Resize queue
            self._q = self._q[self._head:] + self._q[:self._head]
            self._head, self._tail = 0, self._size
            self._q += [None] * (len(self._q) * (CircularQueue.RESIZE_FACTOR - 1)) 
        self._q[self._tail] = item
        self._tail = (self._tail + 1) % len(self._q)
        self._size += 1
    
    def dequeue(self):
        val = self._q[self._head]
        self._head = (self._head + 1) % len(self._q)
        self._size -= 1
        return val
    
    def peek_head(self):
        return self._q[self._head]
    
    def peek_tail(self):
        return self._q[self._tail - 1]

    def size(self):
        return self._size

In [41]:
# Tests
cq = CircularQueue(4)

cq.enqueue(1)
cq.enqueue(2)
cq.enqueue(3)
cq.enqueue(4)
assert cq.size() == 4
assert cq.peek_head() == 1
assert cq.peek_tail() == 4

cq.dequeue()
cq.dequeue()
assert cq.peek_head() == 3
assert cq.peek_tail() == 4
cq.enqueue(5)
cq.enqueue(6)
assert cq.peek_head() == 3
assert cq.peek_tail() == 6
assert cq.size() == 4

cq.enqueue(7)
assert cq.size() == 5

Time Complexity:
* enqueue() - amortized `O(1)`
* dequeue() - `O(1)`

Space: `O(n)`

## 8.8 Implement a Queue using Stacks

In [59]:
class QueueStack:
    """
    Queue wrapper over a Stack
    """
    # Constructor
    def __init__(self):
        self._eq = Stack()
        self._dq = Stack()
    
    # Methods
    def enqueue(self, item):
        self._eq.push(item)
    
    def dequeue(self):
        if self._dq.size() == 0:
            self._eq, self._dq = self._move_stacks(self._eq, self._dq)
        return self._dq.pop()
    
    # Functions
    def _move_stacks(self, s1, s2):
        while s1.size() != 0:
            s2.push(s1.pop())
        return s1, s2

In [62]:
# Tests
qs = QueueStack()
qs.enqueue(1)
qs.enqueue(2)
assert qs.dequeue() == 1
assert qs.dequeue() == 2